# LLM Fine Tuning - Quantization 입문

## 학습 목표
1. Quantization의 개념과 필요성 이해
2. Quantization의 작동 원리 학습
3. EXAONE 모델을 활용한 실습
4. 한국어 데이터셋을 이용한 Fine Tuning 경험

---

## 목차
1. [Quantization이란 무엇인가?](#1-quantization이란-무엇인가)
2. [왜 Quantization이 필요한가?](#2-왜-quantization이-필요한가)
3. [Quantization의 종류](#3-quantization의-종류)
4. [실습: EXAONE 모델 Quantization](#4-실습-exaone-모델-quantization)
5. [한국어 데이터셋으로 Fine Tuning](#5-한국어-데이터셋으로-fine-tuning)
6. [성능 비교 및 분석](#6-성능-비교-및-분석)


## 1. Quantization이란 무엇인가?

### 기본 개념
**Quantization(양자화)**은 신경망 모델의 가중치(weights)와 활성화(activations)를 더 적은 비트로 표현하는 기술입니다.

### 간단한 예시
- **일반적인 경우**: 32비트 부동소수점 (FP32)
  - 예: 3.14159265... → 32비트로 표현
- **Quantization 후**: 8비트 정수 (INT8)  
  - 예: 3.14159265... → 3 (8비트로 표현)

### 핵심 아이디어
```
높은 정밀도 (FP32) → 낮은 정밀도 (INT8, INT4)
```

**장점**: 메모리 사용량 감소, 연산 속도 향상  
**단점**: 정확도 약간 손실 가능


## 2. 왜 Quantization이 필요한가?

### 현실적인 문제들

#### 문제 1: 메모리 부족
```
7B 모델 (FP32) = 약 28GB 메모리 필요
7B 모델 (INT8) = 약 7GB 메모리 필요  (4배 절약!)
7B 모델 (INT4) = 약 3.5GB 메모리 필요 (8배 절약!)
```

#### 문제 2: 느린 추론 속도
- GPU 메모리 부족으로 CPU 사용 → 매우 느림
- 큰 모델일수록 연산량 증가

#### 문제 3: 비용 문제
- 클라우드 GPU 비용 (A100 80GB: 시간당 $3-4)
- 개인용 GPU 구매 비용 (RTX 4090: 약 200만원)

### Quantization의 해결책

#### 메모리 효율성
| 정밀도 | 메모리 사용량 | 7B 모델 크기 |
|--------|---------------|---------------|
| FP32   | 100%          | 28GB         |
| FP16   | 50%           | 14GB         |
| INT8   | 25%           | 7GB          |
| INT4   | 12.5%         | 3.5GB        |

#### 속도 향상
- INT8 연산이 FP32보다 2-4배 빠름
- 메모리 대역폭 효율성 증가
- 배치 처리 성능 향상


## 3. Quantization의 종류

### 3.1 Post-Training Quantization (PTQ)
- **설명**: 이미 훈련된 모델을 바로 양자화
- **장점**: 빠르고 간단함
- **단점**: 정확도 손실이 클 수 있음

### 3.2 Quantization-Aware Training (QAT)  
- **설명**: 훈련 과정에서 양자화를 고려
- **장점**: 정확도 손실 최소화
- **단점**: 훈련 시간이 오래 걸림

### 3.3 주요 Quantization 방법들

#### BitsAndBytes (8bit, 4bit)
```python
# 8bit 양자화
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True
)

# 4bit 양자화  
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True
)
```

#### GPTQ (Gradient-based Post-Training Quantization)
- GPU에서 빠른 추론
- 4bit 양자화 지원

#### AWQ (Activation-aware Weight Quantization)
- 활성화를 고려한 가중치 양자화
- 더 나은 정확도 유지


## 4. 실습: EXAONE 모델 Quantization

이제 실제로 LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct 모델을 사용해서 Quantization을 실습해보겠습니다.


### 4.1 환경 설정 및 라이브러리 설치


In [ ]:
# 필요한 라이브러리 설치
%pip install transformers accelerate bitsandbytes datasets torch
%pip install peft trl
%pip install sentencepiece protobuf

# GPU 메모리 확인
import torch
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")


### 4.2 EXAONE 모델 로드 (일반 FP16 vs Quantized 비교)


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import time
import psutil
import os

# 모델 이름
model_name = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"

# 토크나이저 로드
print("토크나이저 로드 중...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("토크나이저 로드 완료!")


In [ ]:
# 메모리 사용량 확인 함수
def get_memory_usage():
    """현재 GPU와 RAM 메모리 사용량을 반환"""
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.memory_allocated() / 1024**3  # GB
        gpu_memory_cached = torch.cuda.memory_reserved() / 1024**3  # GB
    else:
        gpu_memory = 0
        gpu_memory_cached = 0
    
    ram_memory = psutil.Process(os.getpid()).memory_info().rss / 1024**3  # GB
    
    return {
        "GPU 메모리 (사용중)": f"{gpu_memory:.2f} GB",
        "GPU 메모리 (캐시포함)": f"{gpu_memory_cached:.2f} GB", 
        "RAM 메모리": f"{ram_memory:.2f} GB"
    }

# 초기 메모리 상태
print("초기 메모리 상태:")
for key, value in get_memory_usage().items():
    print(f"  {key}: {value}")


In [ ]:
# 1. 8bit Quantization 설정
print("=== 8bit Quantization 모델 로드 ===")
start_time = time.time()

# BitsAndBytesConfig for 8bit
bnb_config_8bit = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16
)

try:
    # 8bit 모델 로드
    model_8bit = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config_8bit,
        device_map="auto",
        torch_dtype=torch.float16,
        trust_remote_code=True
    )
    
    load_time_8bit = time.time() - start_time
    print(f"8bit 모델 로드 완료! 소요시간: {load_time_8bit:.2f}초")
    
    # 메모리 사용량 확인
    print("8bit 모델 로드 후 메모리 상태:")
    for key, value in get_memory_usage().items():
        print(f"  {key}: {value}")
        
except Exception as e:
    print(f"8bit 모델 로드 실패: {e}")
    model_8bit = None


In [ ]:
# 2. 4bit Quantization 설정 (더 극단적인 양자화)
print("\n=== 4bit Quantization 모델 로드 ===")
start_time = time.time()

# BitsAndBytesConfig for 4bit  
bnb_config_4bit = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",  # Normal Float 4bit
    bnb_4bit_use_double_quant=True  # Double quantization
)

try:
    # 기존 모델 메모리 해제
    if 'model_8bit' in locals() and model_8bit is not None:
        del model_8bit
        torch.cuda.empty_cache()
    
    # 4bit 모델 로드
    model_4bit = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config_4bit,
        device_map="auto", 
        torch_dtype=torch.float16,
        trust_remote_code=True
    )
    
    load_time_4bit = time.time() - start_time
    print(f"4bit 모델 로드 완료! 소요시간: {load_time_4bit:.2f}초")
    
    # 메모리 사용량 확인
    print("4bit 모델 로드 후 메모리 상태:")
    for key, value in get_memory_usage().items():
        print(f"  {key}: {value}")
        
except Exception as e:
    print(f"4bit 모델 로드 실패: {e}")
    model_4bit = None


### 4.3 Quantized 모델 테스트


In [ ]:
# 테스트용 프롬프트 (한국어)
test_prompt = """아래는 사용자의 질문에 친절하고 정확하게 답변하는 AI 어시스턴트입니다.

질문: 인공지능이 우리 생활에 미치는 긍정적인 영향에 대해 설명해주세요.

답변:"""

# 토크나이징
def generate_response(model, prompt, max_length=200):
    """모델로부터 응답 생성"""
    inputs = tokenizer(prompt, return_tensors="pt")
    
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}
    
    # 추론 시간 측정
    start_time = time.time()
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    
    generation_time = time.time() - start_time
    
    # 디코딩
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return response, generation_time

print("테스트 프롬프트:")
print(test_prompt)


In [ ]:
# 4bit 모델로 추론 테스트
if 'model_4bit' in locals() and model_4bit is not None:
    print("=== 4bit Quantized 모델 추론 테스트 ===")
    
    response_4bit, time_4bit = generate_response(model_4bit, test_prompt)
    
    print(f"추론 시간: {time_4bit:.2f}초")
    print(f"응답 길이: {len(response_4bit)}자")
    print("\n생성된 응답:")
    print("=" * 50)
    print(response_4bit)
    print("=" * 50)
    
    # 메모리 사용량 
    print("\n추론 후 메모리 상태:")
    for key, value in get_memory_usage().items():
        print(f"  {key}: {value}")
else:
    print("4bit 모델이 로드되지 않았습니다.")


## 5. 한국어 데이터셋으로 Fine Tuning

이제 Quantized 모델을 한국어 데이터셋으로 Fine Tuning 해보겠습니다. PEFT(Parameter Efficient Fine Tuning) 기법인 LoRA를 사용하겠습니다.


### 5.1 한국어 데이터셋 준비


In [ ]:
# 한국어 질문-답변 데이터셋 생성
korean_dataset = [
    {
        "instruction": "인공지능의 발전이 우리 사회에 미치는 영향에 대해 설명해주세요.",
        "output": "인공지능의 발전은 우리 사회에 다양한 긍정적인 변화를 가져오고 있습니다. 의료 분야에서는 정확한 진단과 맞춤형 치료가 가능해지고, 교육 분야에서는 개인화된 학습 경험을 제공할 수 있습니다. 또한 자동화를 통해 업무 효율성이 크게 향상되고 있습니다."
    },
    {
        "instruction": "기후변화 문제를 해결하기 위한 개인의 실천 방안을 제시해주세요.",
        "output": "기후변화 문제 해결을 위해 개인이 실천할 수 있는 방안들이 있습니다. 에너지 절약을 위해 불필요한 전기 사용을 줄이고, 대중교통이나 자전거 이용을 늘리며, 재활용을 적극적으로 실천하고, 친환경 제품을 선택하는 것이 중요합니다."
    },
    {
        "instruction": "건강한 식습관의 중요성과 실천 방법을 알려주세요.",
        "output": "건강한 식습관은 질병 예방과 삶의 질 향상에 매우 중요합니다. 규칙적인 식사 시간을 지키고, 다양한 영양소를 균형 있게 섭취하며, 충분한 수분 섭취와 함께 가공식품을 줄이고 신선한 재료로 만든 음식을 선택하는 것이 좋습니다."
    },
    {
        "instruction": "효과적인 시간 관리 방법에 대해 조언해주세요.",
        "output": "효과적인 시간 관리를 위해서는 우선순위를 명확히 정하고, 할 일 목록을 작성하여 체계적으로 관리하는 것이 중요합니다. 또한 집중할 수 있는 환경을 만들고, 적절한 휴식을 취하며, 불필요한 일들을 과감히 줄이는 것도 필요합니다."
    },
    {
        "instruction": "독서의 장점과 올바른 독서 습관에 대해 설명해주세요.",
        "output": "독서는 지식 확장, 어휘력 향상, 창의력 개발, 스트레스 해소 등 다양한 장점이 있습니다. 올바른 독서 습관을 위해서는 매일 일정한 시간을 독서에 할애하고, 다양한 장르의 책을 읽으며, 읽은 내용에 대해 생각하고 기록하는 것이 좋습니다."
    }
]

print("한국어 학습 데이터셋 준비 완료!")
print(f"총 {len(korean_dataset)}개의 데이터")
print("\n첫 번째 예시:")
print(f"질문: {korean_dataset[0]['instruction']}")
print(f"답변: {korean_dataset[0]['output']}")


### 5.2 LoRA 설정 및 Fine Tuning 준비


In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import TrainingArguments, Trainer
from datasets import Dataset

# LoRA 설정
lora_config = LoraConfig(
    r=16,                 # rank: LoRA의 차원 (작을수록 메모리 절약)
    lora_alpha=32,        # LoRA scaling parameter
    target_modules=[      # LoRA를 적용할 레이어들
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.1,     # dropout 비율
    bias="none",          # bias 처리 방식
    task_type="CAUSAL_LM" # 태스크 타입
)

print("LoRA 설정 완료!")
print(f"LoRA rank: {lora_config.r}")
print(f"Target modules: {lora_config.target_modules}")

# 4bit 모델을 LoRA 학습용으로 준비
if 'model_4bit' in locals() and model_4bit is not None:
    model_4bit = prepare_model_for_kbit_training(model_4bit)
    model_4bit = get_peft_model(model_4bit, lora_config)
    
    # 학습 가능한 파라미터 확인
    model_4bit.print_trainable_parameters()
    print("LoRA 모델 준비 완료!")
else:
    print("4bit 모델이 없어서 LoRA 설정을 건너뜁니다.")


In [ ]:
# 데이터 전처리 함수
def format_instruction(example):
    """Instruction 형태로 데이터 포맷팅"""
    prompt = f"""아래는 사용자의 질문에 친절하고 정확하게 답변하는 AI 어시스턴트입니다.

질문: {example['instruction']}

답변: {example['output']}"""
    return {"text": prompt}

# 데이터셋을 Hugging Face Dataset으로 변환
train_dataset = Dataset.from_list(korean_dataset)
train_dataset = train_dataset.map(format_instruction)

print("데이터 전처리 완료!")
print(f"학습 데이터 개수: {len(train_dataset)}")
print("\n전처리된 첫 번째 예시:")
print(train_dataset[0]['text'][:200] + "...")


### 5.3 Fine Tuning 실행


In [ ]:
# 데이터 콜레이터 함수
def data_collator(examples):
    """배치 데이터 처리"""
    texts = [example['text'] for example in examples]
    
    # 토크나이징
    tokenized = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    
    # labels는 input_ids와 동일 (언어 모델링)
    tokenized['labels'] = tokenized['input_ids'].clone()
    
    return tokenized

# 학습 설정
training_args = TrainingArguments(
    output_dir="./korean-exaone-lora",      # 저장 디렉토리
    num_train_epochs=1,                      # 에포크 수 (데모용으로 1로 설정)
    per_device_train_batch_size=1,           # 배치 크기 (메모리 절약)
    gradient_accumulation_steps=4,           # 그래디언트 누적 스텝
    learning_rate=2e-4,                      # 학습률
    warmup_steps=10,                         # 워밍업 스텝
    logging_steps=1,                         # 로깅 간격
    save_steps=50,                           # 저장 간격
    evaluation_strategy="no",                # 평가 전략
    save_total_limit=3,                      # 저장할 체크포인트 수
    remove_unused_columns=False,             # 사용하지 않는 컬럼 제거하지 않음
    dataloader_pin_memory=False,             # 메모리 핀 비활성화
)

print("학습 설정 완료!")
print(f"배치 크기: {training_args.per_device_train_batch_size}")
print(f"학습률: {training_args.learning_rate}")
print(f"에포크: {training_args.num_train_epochs}")


In [ ]:
# Trainer 생성 및 학습 실행
if 'model_4bit' in locals() and model_4bit is not None:
    print("=== Fine Tuning 시작 ===")
    
    trainer = Trainer(
        model=model_4bit,
        args=training_args,
        train_dataset=train_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer,
    )
    
    # 학습 시작 시간 기록
    training_start_time = time.time()
    
    try:
        # 학습 실행
        trainer.train()
        
        training_time = time.time() - training_start_time
        print(f"\n학습 완료! 소요시간: {training_time:.2f}초")
        
        # 모델 저장
        trainer.save_model()
        print("모델 저장 완료!")
        
        # 학습 후 메모리 상태
        print("\n학습 후 메모리 상태:")
        for key, value in get_memory_usage().items():
            print(f"  {key}: {value}")
            
    except Exception as e:
        print(f"학습 중 오류 발생: {e}")
        print("이는 정상적인 현상일 수 있습니다. (데모 환경의 메모리 제한)")
        
else:
    print("4bit 모델이 없어서 Fine Tuning을 건너뜁니다.")
    print("위의 셀들을 실행해서 모델을 먼저 로드해주세요.")


## 6. 성능 비교 및 분석


In [ ]:
# Fine Tuning 후 모델 테스트
test_questions = [
    "AI 기술의 미래에 대해 어떻게 생각하시나요?",
    "환경 보호를 위해 우리가 할 수 있는 일은 무엇인가요?",
    "효과적인 학습 방법을 추천해주세요."
]

print("=== Fine Tuning 후 모델 성능 테스트 ===")

if 'model_4bit' in locals() and model_4bit is not None:
    for i, question in enumerate(test_questions, 1):
        print(f"\n{i}. 질문: {question}")
        
        # 프롬프트 구성
        test_prompt = f"""아래는 사용자의 질문에 친절하고 정확하게 답변하는 AI 어시스턴트입니다.

질문: {question}

답변:"""
        
        # 응답 생성
        response, gen_time = generate_response(model_4bit, test_prompt, max_length=150)
        
        # 답변 부분만 추출
        answer_start = response.find("답변:") + 3
        answer = response[answer_start:].strip()
        
        print(f"답변: {answer}")
        print(f"생성 시간: {gen_time:.2f}초")
        print("-" * 50)
else:
    print("Fine Tuning된 모델이 없습니다.")


### 6.1 Quantization 효과 정리


In [ ]:
# Quantization 효과 요약
print("=== Quantization 효과 분석 ===")
print()

# 이론적 메모리 절약 효과
model_size_fp16 = 7.8 * 2  # 7.8B parameters * 2 bytes (FP16)
model_size_8bit = 7.8 * 1  # 7.8B parameters * 1 byte (INT8)
model_size_4bit = 7.8 * 0.5  # 7.8B parameters * 0.5 bytes (INT4)

print("📊 메모리 사용량 비교 (이론적):")
print(f"  FP16:  {model_size_fp16:.1f} GB")
print(f"  8bit:  {model_size_8bit:.1f} GB ({model_size_fp16/model_size_8bit:.1f}배 절약)")
print(f"  4bit:  {model_size_4bit:.1f} GB ({model_size_fp16/model_size_4bit:.1f}배 절약)")
print()

print("⚡ 추론 속도:")
print("  - 8bit: FP16 대비 1.5-2배 빠름")
print("  - 4bit: FP16 대비 2-3배 빠름")
print()

print("🎯 정확도:")
print("  - 8bit: 원본 대비 95-98% 유지")
print("  - 4bit: 원본 대비 90-95% 유지")
print()

print("💰 비용 절약:")
print("  - GPU 메모리 요구량 감소")
print("  - 클라우드 비용 절약 (더 작은 GPU 사용 가능)")
print("  - 추론 속도 향상으로 처리량 증가")


## 7. 학습 정리 및 다음 단계

### 7.1 오늘 배운 내용

1. **Quantization 개념**
   - 신경망 모델의 가중치를 더 적은 비트로 표현하는 기술
   - FP32 → FP16 → INT8 → INT4 순으로 메모리 절약

2. **Quantization의 필요성**
   - 메모리 부족 문제 해결
   - 추론 속도 향상
   - 비용 절약

3. **실습 내용**
   - EXAONE 7.8B 모델을 4bit로 양자화
   - 한국어 데이터셋으로 LoRA Fine Tuning
   - 메모리 사용량과 성능 비교

### 7.2 다음 단계 학습 방향

1. **고급 Quantization 기법**
   - GPTQ, AWQ 등 다른 양자화 방법
   - Mixed-precision 학습

2. **더 복잡한 Fine Tuning**
   - 대규모 데이터셋 활용
   - 다양한 PEFT 기법 (AdaLoRA, QLoRA 등)

3. **모델 배포**
   - Quantized 모델의 실제 서비스 적용
   - ONNX, TensorRT 등 최적화 도구 활용

### 7.3 추천 자료

- Hugging Face Transformers 문서
- PEFT 라이브러리 튜토리얼
- BitsAndBytes 공식 문서
- 최신 Quantization 연구 논문들
